In [1]:
# ==================== 单元格 1: 环境准备与库导入 ====================

# ---- 第一部分：为Binder在线环境安装中文字体 (关键步骤) ----
# 这段代码仅在Linux环境（如Binder）中运行，在Windows/macOS本地运行时会自动跳过。
import sys
import subprocess

def install_if_missing(package):
    """检查并安装系统软件包"""
    try:
        # 尝试执行一个不改变系统的检查命令
        subprocess.run(['dpkg', '-l', package], check=True, capture_output=True)
        print(f"✅ 系统已安装字体包: {package}")
    except subprocess.CalledProcessError:
        # 如果未找到，则安装
        print(f"🔧 Binder环境中缺少中文字体，正在安装 {package}...")
        subprocess.run(['apt-get', 'update'], capture_output=True)
        result = subprocess.run(['apt-get', 'install', '-y', package], 
                               capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ 字体包 {package} 安装成功")
        else:
            print(f"⚠️  字体包安装可能遇到问题，但将继续运行: {result.stderr[:100]}")

# 安装中文字体包（文泉驿正黑）
if 'linux' in sys.platform:
    install_if_missing('fonts-wqy-zenhei')
else:
    print("ℹ️  非Linux环境，跳过系统字体安装。")

print("✅ 环境与库准备完成，中文字体已配置。")
print("-" * 60)

ℹ️  非Linux环境，跳过系统字体安装。
✅ 环境与库准备完成，中文字体已配置。
------------------------------------------------------------


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 生成数据（修正参数）
print("正在生成数据...")
np.random.seed(42)

# 修正方案1：增加特征数但只使用前2个
X, y = make_classification(
    n_samples=80, 
    n_features=3,           # 改为3个特征
    n_informative=2,        # 信息特征为2
    n_redundant=0,          # 冗余特征为0
    n_repeated=0,           # 重复特征为0
    n_classes=3,
    n_clusters_per_class=1, 
    random_state=42
)

# 只取前两个特征用于可视化
X = X[:, :2]

# 归一化到[0,1]范围
X_min, X_max = X.min(0), X.max(0)
X = (X - X_min) / (X_max - X_min)

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

print(f"✅ 数据生成完成！共{len(X)}个样本，3个类别")

# 创建控件
k_slider = widgets.IntSlider(
    value=5, min=1, max=20, step=1,
    description='K值:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='95%')
)

x_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.01,
    description='X坐标:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='95%')
)

y_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.01,
    description='Y坐标:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='95%')
)

metric_dropdown = widgets.Dropdown(
    options=[
        ('欧式距离', 'euclidean'), 
        ('曼哈顿距离', 'manhattan'),
        ('切比雪夫距离', 'chebyshev')
    ],
    value='euclidean',
    description='距离度量:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='95%')
)

weighted_check = widgets.Checkbox(
    value=False,
    description='距离加权投票',
    style={'description_width': 'initial'}
)

random_btn = widgets.Button(
    description='🎲 随机测试点',
    button_style='info',
    tooltip='生成随机测试点',
    layout=widgets.Layout(width='95%')
)

reset_btn = widgets.Button(
    description='🔄 重新生成数据',
    button_style='warning',
    tooltip='重新生成训练数据',
    layout=widgets.Layout(width='95%')
)

# 输出区域
output = widgets.Output()

def generate_new_data():
    """生成新的数据集"""
    global X, y
    
    # 使用不同的随机种子
    new_seed = np.random.randint(1000)
    
    # 生成新数据
    X_new, y_new = make_classification(
        n_samples=80, 
        n_features=3,
        n_informative=2,
        n_redundant=0,
        n_repeated=0,
        n_classes=3,
        n_clusters_per_class=1, 
        random_state=new_seed
    )
    
    X_new = X_new[:, :2]
    X_new = (X_new - X_new.min(0)) / (X_new.max(0) - X_new.min(0))
    
    X, y = X_new, y_new
    
    with output:
        clear_output(wait=True)
        print(f"✅ 已生成新数据集 (随机种子: {new_seed})")
        print(f"📊 类别分布: 类别0={sum(y==0)}个, 类别1={sum(y==1)}个, 类别2={sum(y==2)}个")

def visualize_knn(k, test_x, test_y, metric, weighted):
    with output:
        clear_output(wait=True)
        
        # 准备测试点
        test_point = np.array([[test_x, test_y]])
        
        # 训练KNN
        weights = 'distance' if weighted else 'uniform'
        knn = KNeighborsClassifier(
            n_neighbors=k, 
            metric=metric,
            weights=weights
        )
        knn.fit(X, y)
        
        # 预测
        prediction = knn.predict(test_point)[0]
        probabilities = knn.predict_proba(test_point)[0]
        
        # 获取邻居
        distances, indices = knn.kneighbors(test_point)
        neighbors_X = X[indices[0]]
        neighbors_y = y[indices[0]]
        
        # 创建图形
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # ========== 左图：分类可视化 ==========
        for i in range(3):
            mask = y == i
            ax1.scatter(X[mask, 0], X[mask, 1],
                       c=colors[i], label=f'类别 {i}',
                       alpha=0.6, s=50, edgecolors='white', linewidth=0.5)
        
        # 邻居连线（最多8个）
        max_neighbors = min(k, 8)
        for j in range(max_neighbors):
            nx, ny = neighbors_X[j]
            # 连线
            ax1.plot([test_x, nx], [test_y, ny],
                    'gray', alpha=0.5, linestyle='--', linewidth=1)
            
            # 高亮邻居点
            ax1.scatter(nx, ny, 
                       c=colors[neighbors_y[j]], 
                       s=80, edgecolors='yellow', linewidth=1.5,
                       alpha=0.8)
            
            # 距离标签（只显示前3个）
            if j < 3:
                mid_x = (test_x + nx) / 2
                mid_y = (test_y + ny) / 2
                ax1.text(mid_x, mid_y + 0.02, 
                        f'{distances[0][j]:.2f}',
                        fontsize=9, ha='center',
                        bbox=dict(boxstyle="round,pad=0.2", 
                                 facecolor="white", alpha=0.7))
        
        # 测试点
        ax1.scatter(test_x, test_y,
                   c=colors[prediction], s=200, marker='*',
                   edgecolors='black', linewidth=2,
                   label=f'测试点 (预测: {prediction})')
        
        ax1.set_xlim(-0.1, 1.1)
        ax1.set_ylim(-0.1, 1.1)
        ax1.set_xlabel('特征 1', fontsize=12)
        ax1.set_ylabel('特征 2', fontsize=12)
        ax1.set_title(f'KNN分类可视化 (K={k}, {metric}距离)', fontsize=14, pad=15)
        ax1.legend(loc='upper right', fontsize=10)
        ax1.grid(True, alpha=0.3)
        
        # ========== 右图：投票统计 ==========
        unique, counts = np.unique(neighbors_y, return_counts=True)
        vote_counts = [0, 0, 0]
        for cls, count in zip(unique, counts):
            vote_counts[cls] = count
        
        # 如果是加权投票，计算加权票数
        if weighted:
            weights_array = 1.0 / (distances[0] + 1e-10)
            weighted_votes = [0, 0, 0]
            for j, cls in enumerate(neighbors_y):
                weighted_votes[cls] += weights_array[j]
            display_votes = weighted_votes
            ylabel = "加权票数"
            bar_text = [f'{v:.2f}' for v in display_votes]
        else:
            display_votes = vote_counts
            ylabel = "票数"
            bar_text = [str(int(v)) for v in display_votes]
        
        # 绘制条形图
        bars = ax2.bar(['类别 0', '类别 1', '类别 2'],
                      display_votes, color=colors, alpha=0.7)
        
        # 添加数值标签
        for bar, val, txt in zip(bars, display_votes, bar_text):
            height = bar.get_height()
            if height > 0:
                ax2.text(bar.get_x() + bar.get_width()/2.,
                        height + max(display_votes)*0.03, txt,
                        ha='center', va='bottom', fontsize=11)
        
        # 添加概率饼图（子图）
        ax2_inset = ax2.inset_axes([0.6, 0.6, 0.35, 0.35])
        ax2_inset.pie(probabilities, colors=colors, 
                     autopct='%1.1f%%', startangle=90,
                     textprops={'fontsize': 9})
        ax2_inset.set_title('概率分布', fontsize=10)
        
        ax2.set_ylabel(ylabel, fontsize=12)
        ax2.set_title('投票统计', fontsize=14, pad=15)
        ax2.grid(True, alpha=0.3, axis='y')
        
        if max(display_votes) > 0:
            ax2.set_ylim(0, max(display_votes) * 1.3)
        
        plt.tight_layout()
        plt.show()
        
        # ========== 显示结果信息 ==========
        print("=" * 60)
        print("📊 KNN算法执行结果")
        print("=" * 60)
        print(f"📍 测试点坐标: ({test_x:.3f}, {test_y:.3f})")
        print(f"⚙️ 算法参数:")
        print(f"   - K值: {k}")
        print(f"   - 距离度量: {metric}")
        print(f"   - 投票方式: {'加权' if weighted else '普通'}")
        print(f"🏆 预测结果: 类别 {prediction}")
        print(f"📈 置信度: {max(probabilities):.1%}")
        print(f"👥 最近邻分布:")
        print(f"   - 类别 0: {vote_counts[0]}个邻居")
        print(f"   - 类别 1: {vote_counts[1]}个邻居")
        print(f"   - 类别 2: {vote_counts[2]}个邻居")
        
        if weighted:
            print(f"💰 加权票数: 类别0={display_votes[0]:.2f}, 类别1={display_votes[1]:.2f}, 类别2={display_votes[2]:.2f}")
        
        print("=" * 60)

# 回调函数
def on_value_change(change):
    visualize_knn(k_slider.value, x_slider.value,
                 y_slider.value, metric_dropdown.value,
                 weighted_check.value)

def on_random_click(b):
    new_x, new_y = np.random.rand(2)
    x_slider.value = new_x
    y_slider.value = new_y

def on_reset_click(b):
    generate_new_data()
    # 重新可视化
    visualize_knn(k_slider.value, x_slider.value,
                 y_slider.value, metric_dropdown.value,
                 weighted_check.value)

# 绑定事件
k_slider.observe(on_value_change, 'value')
x_slider.observe(on_value_change, 'value')
y_slider.observe(on_value_change, 'value')
metric_dropdown.observe(on_value_change, 'value')
weighted_check.observe(on_value_change, 'value')
random_btn.on_click(on_random_click)
reset_btn.on_click(on_reset_click)

# 创建界面
print("\n" + "="*70)
print("🎯 KNN算法交互式学习系统")
print("="*70)
print("系统功能:")
print("  ✅ 可调节K值（1-20）")
print("  ✅ 三种距离度量方法")
print("  ✅ 普通/加权投票可选")
print("  ✅ 交互式测试点控制")
print("  ✅ 实时可视化分类结果")
print("  ✅ 详细算法执行信息")
print("="*70)

controls = widgets.VBox([
    widgets.HTML("<h3 style='margin-top:0; color:#2c3e50;'>🎮 控制面板</h3>"),
    
    widgets.VBox([
        widgets.Label("邻居数量控制:", style={'fontWeight': 'bold'}),
        k_slider
    ], style={'padding': '10px', 'backgroundColor': '#f0f8ff', 
              'borderRadius': '8px', 'marginBottom': '15px'}),
    
    widgets.VBox([
        widgets.Label("测试点位置控制:", style={'fontWeight': 'bold'}),
        widgets.VBox([  # 修改为VBox，使滑块垂直排列
            widgets.HBox([widgets.Label("X坐标滑块:", style={'fontWeight': 'normal'})]),
            x_slider,
            widgets.HTML("<div style='height:10px;'></div>"),  # 添加间距
            widgets.HBox([widgets.Label("Y坐标滑块:", style={'fontWeight': 'normal'})]),
            y_slider
        ])
    ], style={'padding': '10px', 'backgroundColor': '#f0f8ff', 
              'borderRadius': '8px', 'marginBottom': '15px'}),
    
    widgets.VBox([
        widgets.Label("算法参数:", style={'fontWeight': 'bold'}),
        metric_dropdown,
        weighted_check
    ], style={'padding': '10px', 'backgroundColor': '#f0f8ff', 
              'borderRadius': '8px', 'marginBottom': '15px'}),
    
    widgets.VBox([
        widgets.Label("数据操作:", style={'fontWeight': 'bold'}),
        widgets.HBox([random_btn, reset_btn])
    ], style={'padding': '10px', 'backgroundColor': '#f0f8ff', 
              'borderRadius': '8px', 'marginBottom': '15px'}),
    
    widgets.HTML("""
    <div style='padding:12px; background:#e8f6f3; border-radius:8px;'>
    <b>💡 使用说明:</b>
    <ul style='margin:8px 0; padding-left:20px;'>
    <li><b>K值滑块</b>: 调整邻居数量，观察决策边界变化</li>
    <li><b>坐标滑块</b>: 移动测试点，观察分类结果变化</li>
    <li><b>距离度量</b>: 比较不同距离公式的影响</li>
    <li><b>加权投票</b>: 近距离邻居有更大话语权</li>
    <li><b>随机按钮</b>: 快速探索不同位置</li>
    <li><b>重置按钮</b>: 生成新的训练数据</li>
    </ul>
    </div>
    """)
], layout=widgets.Layout(width='400px', padding='15px',
                        border='2px solid #3498db',
                        borderRadius='10px'))

# 显示
display(widgets.HBox([
    controls,
    widgets.VBox([output], layout=widgets.Layout(width='70%'))
]))

# 初始显示
print("\n正在初始化可视化...")
visualize_knn(k_slider.value, x_slider.value,
             y_slider.value, metric_dropdown.value,
             weighted_check.value)
print("\n✅ 系统就绪！开始探索KNN算法吧！")



正在生成数据...
✅ 数据生成完成！共80个样本，3个类别

🎯 KNN算法交互式学习系统
系统功能:
  ✅ 可调节K值（1-20）
  ✅ 三种距离度量方法
  ✅ 普通/加权投票可选
  ✅ 交互式测试点控制
  ✅ 实时可视化分类结果
  ✅ 详细算法执行信息



正在初始化可视化...

✅ 系统就绪！开始探索KNN算法吧！
